# Limpieza de Datos Dataset Movimientos-20190716

In [1]:
import pandas as pd
from utils_functions import  upload_to_bucket

# Importar Dataset

#### Importar el dataset y observar los primeros registros con head y verificar los tipos de datos con el método info.

In [2]:
movimientos_col_names = [
    'idb', 'id_producto', 'fecha_trans', 'stock_unidades', 'venta_unidades', 'precio_unitario',
    'cd_abast', 'sigue_prod_en_rol'
]

dtypes = {
    'stock_unidades': 'int64',
    'venta_unidades': 'int64'
}

In [3]:
path = '../data/VMI_Movimientos-20190716.txt'
movimientos_df = pd.read_csv(path,
                 delimiter = '\t',
                 header=None,
                 names=movimientos_col_names,
                 dtype=dtypes)

In [4]:
movimientos_df.head()

,idb,id_producto,fecha_trans,stock_unidades,venta_unidades,precio_unitario,cd_abast,sigue_prod_en_rol
0,242,228217,20160715,24,0,13.63,0,SI
1,242,228217,20160716,24,0,13.63,0,SI
2,242,228217,20160717,24,0,13.63,0,SI
3,242,228217,20160718,24,0,13.63,0,SI
4,242,228217,20160719,24,0,13.63,0,SI


In [5]:
movimientos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8347817 entries, 0 to 8347816
Data columns (total 8 columns):
 #   Column             Dtype  
---  ------             -----  
 0   idb                int64  
 1   id_producto        int64  
 2   fecha_trans        int64  
 3   stock_unidades     int64  
 4   venta_unidades     int64  
 5   precio_unitario    float64
 6   cd_abast           int64  
 7   sigue_prod_en_rol  object 
dtypes: float64(1), int64(6), object(1)
memory usage: 509.5+ MB


#### Ajustes al dataframe
- Convertir la columna fecha_trans a tipo de dato datetime.
- Crear columnas de año, mes y día apartir de la columna fecha.
- Convertir la columna sigue_prod_en_rol a tipo de dato string.
- Verificar por valores nulos y repetidos.
- Convertir la columna sigue_prod_en_rol a variable numerica donde 1 corresponde a SI y 0 corresponde a NO.
- Obtener las fechas máximas y minimas de transaciones
- Obtener una análisis descriptivo de los datos.

In [6]:
movimientos_df['fecha_trans'] = pd.to_datetime(movimientos_df['fecha_trans'], format='%Y%m%d')

In [7]:
movimientos_df['year'] = movimientos_df['fecha_trans'].dt.year
movimientos_df['month'] = movimientos_df['fecha_trans'].dt.month
movimientos_df['weekday'] = movimientos_df['fecha_trans'].dt.weekday

- Obtener la cantidad de transacciones por año.

In [10]:
print(f"La cantidad de movimientos por año es:\n{movimientos_df.groupby('year').idb.count()}")

La cantidad de movimientos por año es:
year
2016    1177264
2017    2343400
2018    2914155
2019    1912998
Name: idb, dtype: int64


In [11]:
movimientos_df.isnull().sum(axis=0)

idb                  0
id_producto          0
fecha_trans          0
stock_unidades       0
venta_unidades       0
precio_unitario      0
cd_abast             0
sigue_prod_en_rol    0
year                 0
month                0
weekday              0
dtype: int64

In [12]:
movimientos_df.duplicated().sum()

0

In [21]:
print(f"La cantidad de productos de referencias únicas en el dataset son: {movimientos_df.id_producto.nunique()}")

La cantidad de productos de referencias únicas en el dataset son: 92


In [14]:
# Verificar si existen valores duplicados para el subset: idb, id_producto, fecha_trans, precio_unitario
duplicated_mask = movimientos_df.duplicated(subset=['idb', 'id_producto', 'precio_unitario', 'fecha_trans'], keep='first')
num_duplicated_records = sum(duplicated_mask)
print(f"El número de registros duplicados es: {num_duplicated_records}")

El número de registros duplicados es: 0


In [15]:
movimientos_df['sigue_prod_en_rol'] = movimientos_df['sigue_prod_en_rol'].str.lower().apply(lambda x: 1 if x == 'si' else 0)

In [16]:
max_date = movimientos_df['fecha_trans'].max()
min_date = movimientos_df['fecha_trans'].min()

print(f"Minimum date: {min_date}")
print(f"Maximum date: {max_date}")

Minimum date: 2016-07-15 00:00:00
Maximum date: 2019-07-14 00:00:00


In [17]:
movimientos_df.describe()

,idb,id_producto,stock_unidades,venta_unidades,precio_unitario,cd_abast,sigue_prod_en_rol,year,month,weekday
count,8.347817e+06,8.347817e+06,8.347817e+06,8.347817e+06,8.347817e+06,8.347817e+06,8.347817e+06,8.347817e+06,8.347817e+06,8.347817e+06
mean,6.988555e+03,3.785489e+05,2.065585e+01,3.133696e-01,6.426050e+01,5.435286e+03,9.609797e-01,2.017666e+03,6.485551e+00,3.002297e+00
std,3.300353e+03,9.159373e+04,1.488001e+02,8.950359e-01,6.582745e+01,4.533729e+03,1.936432e-01,9.811683e-01,3.454577e+00,2.000007e+00
min,2.420000e+02,1.641460e+05,-5.420000e+02,-4.500000e+01,0.000000e+00,0.000000e+00,0.000000e+00,2.016000e+03,1.000000e+00,0.000000e+00
25%,5.208000e+03,2.929450e+05,6.000000e+00,0.000000e+00,2.251000e+01,0.000000e+00,1.000000e+00,2.017000e+03,3.000000e+00,1.000000e+00
50%,9.026000e+03,4.152680e+05,1.000000e+01,0.000000e+00,3.823000e+01,9.217000e+03,1.000000e+00,2.018000e+03,6.000000e+00,3.000000e+00
75%,9.159000e+03,4.583930e+05,1.700000e+01,0.000000e+00,7.673000e+01,9.217000e+03,1.000000e+00,2.018000e+03,1.000000e+01,5.000000e+00
max,9.977000e+03,5.113260e+05,1.621800e+04,6.120000e+02,4.238100e+02,9.217000e+03,1.000000e+00,2.019000e+03,1.200000e+01,6.000000e+00


In [18]:
movimientos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8347817 entries, 0 to 8347816
Data columns (total 11 columns):
 #   Column             Dtype         
---  ------             -----         
 0   idb                int64         
 1   id_producto        int64         
 2   fecha_trans        datetime64[ns]
 3   stock_unidades     int64         
 4   venta_unidades     int64         
 5   precio_unitario    float64       
 6   cd_abast           int64         
 7   sigue_prod_en_rol  int64         
 8   year               int64         
 9   month              int64         
 10  weekday            int64         
dtypes: datetime64[ns](1), float64(1), int64(9)
memory usage: 700.6 MB


In [19]:
# Verificar cuantos valores negativos tenemos en la columna venta_unidades
ventas_unidades_negativas = movimientos_df[movimientos_df['venta_unidades'] < 0]
print(f"Las ventas en unidades con valores negativos son en total: {ventas_unidades_negativas.id_producto.count()}")

Las ventas en unidades con valores negativos son en total: 580


In [20]:
# Verificar cuantos valores negativos tenemos en la columna stock unidades
stock_unidades_negativas = movimientos_df[movimientos_df['stock_unidades'] < 0]
print(f"El total de inventario en unidades negativos son en total: {stock_unidades_negativas.id_producto.count()}")

El total de inventario en unidades negativos son en total: 45483


#### Observaciones dataset Movimientos-20190716

1. El conjunto de datos contiene información sobre los movimientos de productos desde el 15 de julio de 2016 hasta el 14 de julio de 2019.
2. El número de registros por año es el siguiente:
        2016: 1,177,264
        2017: 2,343,400
        2018: 2,914,155
        2019: 1,912,998
3. El conjunto de datos consta de 8 columnas identificadas de la siguiente manera:
        - idb: identificador interno de la ubicación
        - id_producto: identificador interno del producto
        - fecha: fecha de la transacción (un día antes de la fecha actual)
        - stock_unidades: número de unidades de producto en la ubicación
        - venta_unidades: número de unidades vendidas
        - precio_unitario: precio de venta unitario
        - cd_abast: identificador de la ubicación de la tienda que suministra el producto a la ubicación actual.
        - sigue_prod_en_rol: indica si el producto sigue en rol (SI) o no (NO) en la ubicación actual.
4. No se observan valores faltantes o duplicados. Esta es una tabla transaccional, por lo que puede haber múltiples registros para el mismo producto y ubicación con diferentes fechas de transacciones.
5. El valor medio de unidades de inventario es de 21 unidades.
6. El valor medio de unidades vendidas es de 0.31 unidades.
7. El valor máximo de ventas es de 612 unidades.
8. El valor mínimo de ventas es de -40 unidades.
9. La media del precio unitario es de 64.26 pesos.
10. Se observan 580 registros que contienen valores negativos en la columna venta_unidades.
11. Se observan 45,483 registros que contienen valores negativos en la columna stock_unidades.
12. La cantidad de productos únicos en el conjunto de datos es de 92.



### Guardar el dataset limpio

In [23]:
movimientos_df.to_csv('../clean_data/movimientos_clean.csv', index=False)

#### Subir csv a google cloud storage

In [24]:
upload_to_bucket('deodorants', '../clean_data/movimientos_clean.csv', 'movimientos')

File ../clean_data/movimientos_clean.csv uploaded successfully to bucket deodorants/movimientos
